# Minigame 12: Patched Evaluation

In this exploration, we're looking at the ability to take policies trained on small patches and to apply them to larger meshes by amalgamating the logits from each of the NN evaluations.

In [1]:
import math
from math import sin,cos
import random

In [2]:
import gym
from gym import spaces, utils
import numpy as np
import ray
import ray.rllib.agents.ppo as ppo

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from glvis import glvis, to_stream
from ipywidgets import Layout

In [4]:
import matplotlib.pyplot as plt

In [5]:
from mfem import path
import mfem.ser as mfem

Define some synthetic test functions.

In [6]:
def rotate(x,theta):
    x0 = x[0]
    y0 = x[1]
    x1 = x0*cos(theta)-y0*sin(theta)
    y1 = x0*sin(theta)+y0*cos(theta)
    return [x1,y1]

In [7]:
def bump(x):
    rsq = x[0]**2 +x[1]**2
    return math.exp(-rsq)

In [8]:
def smooth_step(x):
    return 0.5*(1.0 +math.tanh(x[0]))

In [9]:
def rotated_smooth_step(x,theta):
    xr = rotate(x,theta)
    return smooth_step(xr)

Create classes where we can set the parameters and then eval a bunch of points.

In [10]:
class Step(mfem.PyCoefficient):
    
    def SetParams(self):
        self.theta = random.uniform(0.0, 2.0*math.pi)
        self.dx = [random.uniform(-1.0, 1.0),random.uniform(-1.0, 1.0)]
        
    def EvalValue(self, x):
        return rotated_step(x+self.dx, self.theta)

In [11]:
class Bump(mfem.PyCoefficient):
    
    def SetParams(self):
        self.width = random.uniform(0.05,1.0)
        self.xc = [0.5,0.5]
        self.dx = [random.uniform(-0.5, 0.5),random.uniform(-0.5, 0.5)]

    def EvalValue(self, x):
        return bump((x-self.xc+self.dx)/self.width)

In [12]:
class SmoothStep(mfem.PyCoefficient):
    
    def SetParams(self):
        self.width = random.uniform(5.0, 15.0)
        self.xc = [0.5,0.5]
        self.dx = random.uniform(-0.5,0.5)
        self.theta = random.uniform(0.0, 2.0*math.pi)

    def EvalValue(self, x):
        x -= self.xc
        x += self.dx
        return rotated_smooth_step(x*self.width, self.theta)

In [13]:
class BumpAndSmoothStep(mfem.PyCoefficient):
    
    def SetParams(self):
        self.bump = Bump()
        self.bump.SetParams()
        self.smooth_step = SmoothStep()
        self.smooth_step.SetParams()

    def EvalValue(self, x):
        return 0.5*self.bump.EvalValue(x)+0.5*self.smooth_step.EvalValue(x)

Visualize an instance of the test function. Note that each instance has randomly chosen parameters.  For the steps, it's a rotation angle and a displacement.  For the bumps, it's a width and a displacement.

In [14]:
mesh = mfem.Mesh('inline-quad.mesh')
mesh.UniformRefinement()
mesh.UniformRefinement()
fec = mfem.L2_FECollection(p=1, dim=2)
fes = mfem.FiniteElementSpace(mesh, fec)
u = mfem.GridFunction(fes)
c = Bump()
c.SetParams()
u.ProjectCoefficient(c)

In [15]:
glvis(to_stream(mesh,u) + 'keys Rcjm', 500, 500)

glvis()

Create the gym environment. Note that in this case, this can be just a dummy environment that only serves to define the observation and action spaces for the purposes of evaluation of the policy.

In [16]:
class AMRGameDummy(gym.Env):
        
    # In RLlib, you need the config arg
    def __init__(self,config):
        self.meshfile = 'inline-quad-7.mesh'
        self.mesh = mfem.Mesh(self.meshfile)
        
        # The only reason we need to create a fespace and gf here
        # is to find the sizes needed for the action and observation spaces
        dim = self.mesh.Dimension()
        self.order = 1
        self.fec = mfem.L2_FECollection(self.order, dim)
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)
        self.u = mfem.GridFunction(self.fes);

        # actions are: refine each element, or do nothing
        self.action_space = spaces.Discrete(self.mesh.GetNE())
        
        # observation space: DOFs
        self.observation_space = spaces.Box(-1.0, 1.0, shape=(self.u.Size(),), dtype=np.float32)
        
    def step(self, action):
        pass
    
    def reset(self):
        pass
    
    def render(self):
        pass

Now we want to load a trained policy, and apply it in a strided way.

In [17]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-02-18 14:34:54,980	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '128.15.103.29',
 'raylet_ip_address': '128.15.103.29',
 'redis_address': '128.15.103.29:6379',
 'object_store_address': '/tmp/ray/session_2021-02-18_14-34-54_526415_11249/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-18_14-34-54_526415_11249/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-02-18_14-34-54_526415_11249',
 'metrics_export_port': 61714,
 'node_id': 'd89b6bc13ac4ee34c5c85462f968450448bed363'}

In [18]:
config = ppo.DEFAULT_CONFIG.copy()
config['framework'] = 'tfe'
agent = ppo.PPOTrainer(config, env=AMRGameDummy)

2021-02-18 14:34:57,421	INFO trainer.py:588 -- Executing eagerly, with eager_tracing=False
2021-02-18 14:34:57,422	INFO trainer.py:618 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=11401) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=11401) Instructions for updating:
(pid=11401) non-resource variables are not supported in the long term
(pid=11400) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=11400) Instructions for updating:
(pid=11400) non-resource variables are not supported in the long term
2021-02-18 14:34:59,597	WARNING ut

Restore a policy

In [19]:
agent.restore("/home/rwa/ray_results/PPO_AMRGame_2021-02-17_18-13-50eomp_6hi/checkpoint_8/checkpoint-8")

2021-02-18 14:34:59,637	INFO trainable.py:329 -- Restored on 128.15.103.29 from checkpoint: /home/rwa/ray_results/PPO_AMRGame_2021-02-17_18-13-50eomp_6hi/checkpoint_8/checkpoint-8
2021-02-18 14:34:59,637	INFO trainable.py:336 -- Current state after restoring: {'_iteration': 8, '_timesteps_total': None, '_time_total': 5470.714422464371, '_episodes_total': 403200}


In [20]:
policy = agent.get_policy()

Now we want to create the larger problem we'll be applying this local indicator on.

In [21]:
mesh = mfem.Mesh('inline-quad-30.mesh')
print(mesh.GetNE())
fec = mfem.L2_FECollection(p=1, dim=2)
fes = mfem.FiniteElementSpace(mesh, fec)
u = mfem.GridFunction(fes)
u0 = mfem.GridFunction(fes)
coeff = Bump()
coeff.SetParams()
u.ProjectCoefficient(coeff)
u0.Assign(u) # save so we can restore later if desired
    
def new_function():
    global mesh, fec, fes, u, u0, coeff
    mesh = mfem.Mesh('inline-quad-30.mesh')
    fec = mfem.L2_FECollection(p=1, dim=2)
    fes = mfem.FiniteElementSpace(mesh, fec)
    u = mfem.GridFunction(fes)
    u0 = mfem.GridFunction(fes)
    coeff = Bump()
    coeff.SetParams()
    u.ProjectCoefficient(coeff)
    u0.Assign(u) # save so we can restore later if desired
    return glvis(to_stream(mesh,u) + 'keys Rcjm', 500, 500)

900


In [22]:
def restore_function():
    global mesh, fec, fes, u
    mesh = mfem.Mesh('inline-quad-30.mesh')
    fec = mfem.L2_FECollection(p=1, dim=2)
    fes = mfem.FiniteElementSpace(mesh, fec)
    u = mfem.GridFunction(fes)
    u.Assign(u0)
    #return glvis(to_stream(mesh,u) + 'keys Rcjm', 500, 500)

Build a map from each element to the elements which consist of the "stencil" around it. Since not every element has a full stencil, use a dictionary that only contains the elements containing full stencils as keys.

In [23]:
def build_stencils(mesh, width):
    els = {}
    nx = math.sqrt(mesh.GetNE())
    dx = 1.0/nx
    dim = mesh.Dimension()
    els = {}
    hw = int(width/2)
    c = mfem.Vector(dim)
    x = mfem.Vector(dim)
    for k in range(0,mesh.GetNE()):
        els[k] = []
        mesh.GetElementCenter(k,c)
        full = True
        for j in range(-hw,hw+1):
            for i in range(-hw,hw+1):
                x[0] = c[0]+i*dx
                x[1] = c[1]+j*dx
                if (x[0] < 0.0): full = False
                if (x[0] > 1.0): full = False
                if (x[1] < 0.0): full = False
                if (x[1] > 1.0): full = False
                pt = [[x[0],x[1]]]
                n, el, ip = mesh.FindPoints(pt)
                els[k].append(el[0])
        if (not full):
            els.pop(k)
    return els

Create a function and build the stencils for it.

In [24]:
new_function()
width=7
els = build_stencils(mesh, width)

Create the local observation mesh into which we will copy the dofs for the purposes of creating an observation vector.

In [25]:
obs_mesh = mfem.Mesh('inline-quad-7.mesh')
obs_fec = mfem.L2_FECollection(p=1, dim=2)
obs_fes = mfem.FiniteElementSpace(obs_mesh, obs_fec)
obs_u = mfem.GridFunction(obs_fes)
glvis((obs_mesh), 400, 400,layout = Layout(width='100%', height='400px'))

glvis(layout=Layout(height='400px', width='100%'))

Also build a 0th order L2 field to look at per-element quantities (like logits or prob dist).

In [26]:
fec0 = mfem.L2_FECollection(p=0, dim=2)
fes0 = mfem.FiniteElementSpace(obs_mesh, fec0)
obs_u0 = mfem.GridFunction(fes0)

Now we need a mapping from the "logical" space of the observation mesh into element ids. This has the same ordering as the stencil elements, so we can form a mapping for the purposes of data transfer from the src mesh to the obs mesh.

In [27]:
def build_map(obs_mesh, width):
    id_map = []
    c = [0.5, 0.5]
    x = [0.0, 0.0]
    dx = 1./width
    hw = int(width/2)
    for j in range(-hw,hw+1):
        for i in range(-hw,hw+1):
            x[0] = c[0]+i*dx
            x[1] = c[1]+j*dx
            pt = [[x[0],x[1]]]
            n, el, ip = obs_mesh.FindPoints(pt)
            id_map.append(el[0])
    return id_map

In [28]:
id_map = build_map(obs_mesh, width)

Create a function to transfer from the stencil associated with a src element k into the observation gf.

In [29]:
def transfer_stencil(k):
    global obs_u
    for n in range(len(els[k])):
        dst_el = id_map[n]
        src_el = els[k][n]
        #print("el %d -> el %d" % (src_el,dst_el))
        src_dofs = fes.GetElementDofs(src_el)
        dst_dofs = obs_fes.GetElementDofs(dst_el)
        for d in range(len(src_dofs)):
            obs_u[dst_dofs[d]] = u[src_dofs[d]]

Visualize the observation mesh and function.

In [30]:
def show_obs():
    return glvis(to_stream(obs_mesh,obs_u) + 'keys Rcjm', 500, 500)

Compute the logits for each element in the observation mesh and visualize as a p=0 L2 function.

In [31]:
def show_patch_logits():
    obs = np.array(obs_u.GetDataArray())
    action, _, info = policy.compute_single_action(obs, explore=False)
    print(info)
    logits = np.array(info['action_dist_inputs'],dtype=np.float64)
    obs_u0.Assign(mfem.Vector(logits))
    return glvis(to_stream(obs_mesh,obs_u0) + 'keys Rcjm', 500, 500)

Test it out on a specific src element:

In [55]:
new_function()

glvis()

In [56]:
transfer_stencil(10)
show_obs()

glvis()

In [57]:
show_patch_logits()

{'action_prob': 1.0, 'action_logp': 0.0, 'action_dist_inputs': array([ 0.3983558 ,  0.32411394,  0.02658089,  0.12445277,  0.08604131,
       -0.03915977, -0.505314  , -0.5322843 , -0.18103397,  0.0055941 ,
        0.15430637, -0.08639008, -0.28180975, -0.44558364, -0.17239645,
       -0.59353364, -0.20044664, -0.0784512 ,  0.12802821,  0.02139634,
       -0.10813664, -0.12555745, -0.44509152, -0.20525275, -0.3111641 ,
        0.04017941,  0.23141071,  0.14934148,  0.54135334,  0.49351397,
        0.16484351,  0.20421632,  0.11574204,  0.33057973,  0.43640792,
        0.4813172 ,  0.27212945, -0.10150181, -0.42004895, -0.42518428,
       -0.17669089,  0.17237099, -0.02217911, -0.24495202, -0.080006  ,
        0.15815108,  0.12359463,  0.05708026,  0.22337668], dtype=float32), 'vf_preds': 0.00017622727}


glvis()

In [58]:
transfer_stencil(53)
show_obs()

glvis()

In [59]:
show_patch_logits()

{'action_prob': 1.0, 'action_logp': 0.0, 'action_dist_inputs': array([ 0.3989856 ,  0.3243538 ,  0.02634194,  0.1242981 ,  0.08607035,
       -0.03946233, -0.5063925 , -0.5332473 , -0.1819028 ,  0.00519418,
        0.15367915, -0.08751276, -0.28251183, -0.44678217, -0.17258504,
       -0.5951126 , -0.20068574, -0.07772367,  0.1293352 ,  0.02315509,
       -0.10711592, -0.12499355, -0.44575822, -0.20527342, -0.3112556 ,
        0.04138593,  0.23348556,  0.15063953,  0.54217094,  0.49408188,
        0.16502304,  0.20411669,  0.11535439,  0.3295894 ,  0.4359846 ,
        0.48123002,  0.27153623, -0.10162053, -0.4207048 , -0.42589304,
       -0.17747036,  0.17228012, -0.02202233, -0.24518248, -0.08000149,
        0.15830247,  0.1233688 ,  0.05657959,  0.22316612], dtype=float32), 'vf_preds': 0.00017619644}


glvis()

Iterate over all the elements with full stencils in the src mesh and record 'center' logits for each observation:

In [60]:
pt = [[0.5,0.5]]
n, center_el, ip = obs_mesh.FindPoints(pt)
center_el

[38]

In [71]:
def compute_center_logits(mesh):
    logits = [0.0]*mesh.GetNE()
    for k in els:
        transfer_stencil(k)
        obs = np.array(obs_u.GetDataArray())
        action, _, info = policy.compute_single_action(obs, explore=False)
        obs_logits = info['action_dist_inputs']
        value = info['vf_preds']
        print("value is %f" % value)
        
        # find minimum to use as datum
        #min_logit = 1.e6
        #for j in range(len(id_map)):
        #    src_el = id_map[j]
        #    min_logit = min(min_logit,obs_logits[src_el])
        
        logits[k] = value*obs_logits[center_el][0] # -min_logit
    return logits

In [72]:
logits = compute_center_logits(mesh)
fec0fm = mfem.L2_FECollection(p=0, dim=2)
fes0fm = mfem.FiniteElementSpace(mesh, fec0fm)
log_fm = mfem.GridFunction(fes0)
log_fm.Assign(mfem.Vector(np.array(logits)))
glvis(to_stream(mesh, log_fm), 500, 500)

value is 0.000176
value is 0.000172
value is 0.000174
value is 0.000175
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000175
value is 0.000175
value is 0.000174
value is 0.000171
value is 0.000170
value is 0.000173
value is 0.000173
value is 0.000170
value is 0.000170
value is 0.000173
value is 0.000175
value is 0.000175
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000177
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000177
value is 0.000177
value is 0.000179
value is 0.000179
value is 0.000177
value is 0.000177
value is 0.000178
value is 0.000177
value is 0.000175
value is 0.000176
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0

value is 0.000178
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000177
value is 0.000176
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000177
value is 0.000178
value is 0.000178
value is 0.000177
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000177
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0.000176
value is 0

glvis()

In [63]:
def compute_avg_logits(mesh):
    logits = [0.0]*mesh.GetNE()
    count = [0]*mesh.GetNE()
    
    # accumulate logit sums
    for k in els:
        transfer_stencil(k)
        obs = np.array(obs_u.GetDataArray())
        action, _, info = policy.compute_single_action(obs, explore=False)
        obs_logits = info['action_dist_inputs']
        
        # find minimum to use as datum
#        min_logit = 1.e6
#        for j in range(len(id_map)):
#            src_el = id_map[j]
#            min_logit = min(min_logit,obs_logits[src_el])

        for j in range(len(id_map)):
            dst_el = els[k][j]
            src_el = id_map[j]
            logits[dst_el] += obs_logits[src_el] #-min_logit
            count[dst_el] += 1
    
    # average
    for idx,val in enumerate(logits):
        logits[idx] /= count[idx]
        if (count[idx] < 16):
            logits[idx] = -3
        #logits[idx] /= obs_mesh.GetNE()

    return logits

In [64]:
logits = compute_avg_logits(mesh)
fec0fm = mfem.L2_FECollection(p=0, dim=2)
fes0fm = mfem.FiniteElementSpace(mesh, fec0fm)
log_fm = mfem.GridFunction(fes0)
log_fm.Assign(mfem.Vector(np.array(logits)))
glvis(to_stream(mesh, log_fm), 500, 500)

glvis()

In [42]:
def compute_min_logits(mesh):
    logits = [100.]*mesh.GetNE()
    
    # choose the max at each element
    for k in els:
        transfer_stencil(k)
        obs = np.array(obs_u.GetDataArray())
        action, _, info = policy.compute_single_action(obs, explore=False)
        #print("action is %d" % action)
        obs_logits = info['action_dist_inputs']
        #print(obs_logits)
        for j in range(len(id_map)):
            dst_el = els[k][j]
            src_el = id_map[j]
            logits[dst_el] = min(logits[dst_el],obs_logits[src_el])
        
    return logits

Re-normalize the collected logits into a probability distribution that sums to 1.

In [43]:
def compute_distribution(mesh, u, method):
    if (method == 1):
        logits = compute_center_logits(mesh)
    elif (method == 2):
        logits = compute_avg_logits(mesh)
    elif (method == 3):
        logits = compute_min_logits(mesh)
    sumexp = 0.0
    dist = [0.0] * mesh.GetNE()
    for k in range(mesh.GetNE()):        
        logit = logits[k]
        sumexp += math.exp(logit)
    for k in range(mesh.GetNE()):
        logit = logits[k]
        dist[k] = math.exp(logit)/sumexp
    return dist

Create a similar function that returns elementwise errors via the dg indicator.

In [44]:
def compute_dg_indicator(mesh, u):
    
    # put the L2 gridfunction into a coefficient so we can project it into H1
    u_disc_coeff = mfem.GridFunctionCoefficient(u)
    h1_fec = mfem.H1_FECollection(p=1, dim=2)
    h1_fes = mfem.FiniteElementSpace(mesh, h1_fec)
    u_h1 = mfem.GridFunction(h1_fes)
    u_h1.ProjectDiscCoefficient(u_disc_coeff, mfem.GridFunction.ARITHMETIC)
    
    # put the H1 smoothed function into a coefficient
    u_h1_coeff = mfem.GridFunctionCoefficient(u_h1)
    
    # create a 0-order L2 field to hold errors
    l2_0_fec = mfem.L2_FECollection(p=0,dim=2)
    l2_0_fes = mfem.FiniteElementSpace(mesh,l2_0_fec)

    # Compute elementwise "errors" between continuous and discontinuous fields
    err_gf = mfem.GridFunction(l2_0_fes);
    u.ComputeElementL2Errors(u_h1_coeff, err_gf);
    
    return np.array(err_gf.GetDataArray())

In [45]:
def compute_actual_error_reduction(mesh, u, u0):
    init_err = u.ComputeL2Error(coeff)
    print("init_err: %f" % init_err)
    delta_elem_err = []
    for k in range(mesh.GetNE()):
        mesh = mfem.Mesh('inline-quad-30.mesh')
        fec = mfem.L2_FECollection(p=1, dim=2)
        fes = mfem.FiniteElementSpace(mesh, fec)
        u = mfem.GridFunction(fes)
        u.Assign(u0)
    
        refine_els = []
        refine_els.append(k)
        mesh.GeneralRefinement(mfem.intArray(refine_els))
        u.FESpace().Update()
        u.Update()
        u.ProjectCoefficient(coeff)
        new_err = u.ComputeL2Error(coeff)
        print("delta for %d is %e" % (k,init_err-new_err))
        delta_elem_err.append(init_err -new_err)

    return delta_elem_err

Given an indicator on each element, refine everything over the threshold

In [46]:
def refine_via_indicator(ind, mesh, u, thresh):
    refine_els = []
    for k,e in enumerate(ind):
        if (e > thresh):
            refine_els.append(k)
    mesh.GeneralRefinement(mfem.intArray(refine_els))
    u.FESpace().Update()
    u.Update()
    return glvis(to_stream(mesh,u) + 'keys Rcjm', 500, 500)

Refine everywhere the policy is over a threshold.

In [47]:
def refine_via_policy_threshold(mesh, u, thresh, method):
    dist = compute_distribution(mesh, u, method)
    return refine_via_indicator(dist, mesh, u, thresh)

Refine everywhere the DG indicator is over a threshold.

In [48]:
def refine_via_dg_threshold(mesh, u, thresh):
    ind = compute_dg_indicator(mesh, u)
    return refine_via_indicator(ind, mesh, u, thresh)

In [49]:
def refine_topk_via_indicator(ind, mesh, u, K):
    refine_els = []
    indsort = np.argsort(ind)[::-1]
    for i in range(K):
        refine_els.append(indsort[i])
    mesh.GeneralRefinement(mfem.intArray(refine_els))
    u.FESpace().Update()
    u.Update()
    return glvis(to_stream(mesh,u) + 'keys Rjm', 400, 400,layout = Layout(width='100%', height='400px'))
 
def refine_topk_via_policy(mesh, u, K, method):
    dist = compute_distribution(mesh, u, method)
    return refine_topk_via_indicator(dist, mesh, u, K)
 
def refine_topk_via_dg(mesh, u, K):
    ind = compute_dg_indicator(mesh, u)
    return refine_topk_via_indicator(ind, mesh, u, K)

def refine_topk_via_delta_elem_err(mesh, u, K):
    return refine_topk_via_indicator(delta_elem_err, mesh, u, K)

In [50]:
new_function()

glvis()

In [51]:
delta_elem_err = compute_actual_error_reduction(mesh, u, u0)

init_err: 0.001019
delta for 0 is 0.000000e+00
delta for 1 is 2.168404e-19
delta for 2 is 4.336809e-18
delta for 3 is 2.168404e-19
delta for 4 is 2.602085e-18
delta for 5 is 2.905662e-17
delta for 6 is 7.121040e-16
delta for 7 is 6.245005e-17
delta for 8 is 1.173974e-15
delta for 9 is 1.299872e-14
delta for 10 is 1.776957e-13
delta for 11 is 1.667741e-14
delta for 12 is 1.193923e-15
delta for 13 is 8.131516e-17
delta for 14 is 4.336809e-18
delta for 15 is 6.461845e-17
delta for 16 is 7.496174e-16
delta for 17 is 1.343305e-14
delta for 18 is 1.165888e-13
delta for 19 is 6.700586e-15
delta for 20 is 4.632406e-14
delta for 21 is 2.478074e-13
delta for 22 is 4.125277e-12
delta for 23 is 7.858352e-13
delta for 24 is 9.784185e-12
delta for 25 is 4.991626e-11
delta for 26 is 4.376779e-10
delta for 27 is 8.967513e-11
delta for 28 is 1.453572e-11
delta for 29 is 1.505448e-12
delta for 30 is 1.805025e-13
delta for 31 is 1.835993e-12
delta for 32 is 1.433403e-11
delta for 33 is 8.732209e-11
delta

KeyboardInterrupt: 

In [ ]:
restore_function()
refine_topk_via_delta_elem_err(mesh, u, 100)

In [ ]:
restore_function()
refine_topk_via_dg(mesh, u, 100)

In [ ]:
restore_function()
method = 1 # center
refine_topk_via_policy(mesh, u, 100, method)

In [ ]:
restore_function()
method = 2 # avg
refine_topk_via_policy(mesh, u, 100, method)

In [ ]:
restore_function()
method = 3 # min
refine_topk_via_policy(mesh, u, 100, method)

In [ ]:
new_function()
refine_topk_via_dg(mesh, u, 100)

In [ ]:
restore_function()
method = 1 # center
refine_topk_via_policy(mesh, u, 100, method)

In [ ]:
restore_function()
method = 2 # avg
refine_topk_via_policy(mesh, u, 100, method)